In [14]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [15]:
# define the file path of the .csv
filepath = '../Datafiniti_Fast_Food_Restaurants_May19.csv'
expectancy_file = '../expectancy.csv'

# read csv into pandas and create a dataframe
fastfood_df = pd.read_csv(filepath)

# pull in team member's .csv data to be uploaded onto database
expectancy_df = pd.read_csv(expectancy_file)

In [16]:
# dropping null value columns to avoid errors 
fastfood_df.dropna(inplace=True)

In [17]:
# view dataframe and look at shape
fastfood_df

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.396290,-104.696990,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.081350,-108.586890,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.091480,-108.554110,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.763690,-104.776710,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.740440,-104.926360,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,AVwd7U0T_7pvs4fz-fS_,2016-05-31T13:33:45Z,2018-12-03T09:04:19Z,1803 Carolina Ave,"American Restaurants,Caterers,Soul Food Restau...",Accommodation & Food Services,Bessemer,US,us/al/bessemer/1803carolinaave/785498836,33.399574,-86.951050,Church's Chicken Of Birmingham,35020,AL,http://www.yellowpages.com/bessemer-al/mip/chu...,http://www.churchs.com
9996,AV1XW1fF3D1zeR_xFKHL,2017-07-18T20:19:50Z,2018-12-02T00:08:36Z,8511 Cantilever Way Ste 105,"Mexican Restaurants,Caterers,Delicatessens,Iri...",Accommodation & Food Services,Raleigh,US,us/nc/raleigh/8511cantileverwayste105/-1596320866,35.884500,-78.737541,Luxury Box,27613-6996,NC,https://www.bbb.org/us/nc/raleigh/profile/spor...,"http://www.theluxuryboxofraleigh.com,http://th..."
9997,AV5UVbkiRxPSIh2RiQxV,2017-09-05T23:05:50Z,2018-12-01T07:38:03Z,2601 Plover Rd,"Carry-out food,Hot dog restaurants,Fast Food R...",Accommodation & Food Services,Plover,US,us/wi/plover/2601ploverrd/-1161002137,44.453180,-89.543170,McDonald's,54467,WI,https://www.yellowpages.com/plover-wi/mip/mcdo...,"http://mcdonalds.com,http://www.mcdonalds.com/..."
9998,AVwc9BxYByjofQCxj_GW,2016-03-31T02:43:12Z,2018-12-01T07:35:22Z,170 E Main St,"Carry-out food,Caterers,Restaurant delivery se...",Accommodation & Food Services,Hendersonville,US,us/tn/hendersonville/170emainst/-66712705,36.308262,-86.607339,Pizza Hut,37075,TN,https://foursquare.com/v/pizza-hut/4c5fffd43a3...,http://www.pizzahut.com


In [18]:
# pull the columns we want to use for our dataframe using double brackets to pull the dataframes [[]]
fastfood_cols_df = fastfood_df[['city', 'name']]

# rename the columns
fastfood_cols_df.columns = ['City', 'Restaurant']

# view the df and look at shape
fastfood_cols_df

,City,Restaurant
0,Greeley,Carl's Jr.
1,Grand Junction,Del Taco
2,Grand Junction,Which Wich
3,Aurora,Chipotle Mexican Grill
4,Denver,Taco Bell
...,...,...
9995,Bessemer,Church's Chicken Of Birmingham
9996,Raleigh,Luxury Box
9997,Plover,McDonald's
9998,Hendersonville,Pizza Hut


In [19]:
# look at the differnt restaurants and their value counts using .value_counts()
fastfood_cols_df['Restaurant'].value_counts()

McDonald's                          759
Subway                              650
SUBWAY                              538
Taco Bell                           524
Pizza Hut                           469
                                   ... 
Emidio & Sons Italian Restaurant      1
Jambalaya Shoppe                      1
Saladfarm                             1
Taco Villa                            1
Big Daddy's Burgers                   1
Name: Restaurant, Length: 1540, dtype: int64

In [20]:
# We want to see how many fastfood restaurants are per city in the dataset.
# groupby the cities using .groupby([]) and use the aggregator function .count() and assign to a new dataframe 'fastfood_df'
fastfood_df = fastfood_cols_df.groupby(['City']).count()

# sort the values in descending order based on the 'Restaurant' column in the current dataframe
fastfood_df = fastfood_df.sort_values('Restaurant', ascending = False)

# reset the index so all data are in their own columns and assign to the current dataframe
fastfood_df = fastfood_df.reset_index()

# rename columns to more meaningful names
fastfood_df.columns = ['city', 'number_fastfood']

# view the dataframe and check the shape
fastfood_df

,city,number_fastfood
0,Columbus,105
1,Indianapolis,94
2,Birmingham,69
3,Cincinnati,60
4,Dayton,59
...,...,...
2782,Noble,1
2783,Flagler,1
2784,Nitro,1
2785,Nine Mile Falls,1


In [21]:
# Create a connection to the database so we can upload our data into the proper tables located in our PostgreSQL server
# The name of our database is 'ucsd_teamproject_ETL' and we must define the path the database using our local pw and login settings
# assing this string to the variable 'connection_string'
connection_string = "postgres:postgres@localhost:5432/ucsd_teamproject_ETL"

# create a connection to the server using create_engine() method and name it 'engine'
engine = create_engine(f'postgresql://{connection_string}')

In [22]:
# check the tables in the database
engine.table_names()

['fastfood', 'expectancy']

In [37]:
# upload our fastfood dataframe to the SQL database table named 'fastfood', connection = engine, append if the rows exist, and don't upload the index 

# NOTE do not run again or it will create double entries
# fastfood_df.to_sql(name='fastfood', con=engine, if_exists='append', index=False)

In [5]:
# upload my team member's data to the database
# upload the expectancy dataframe to the SQL database table named 'expectancy', connection = engine, append if the rows exist, and don't upload the index 

# NOTE do not run again or it will create double entries
# expectancy_df.to_sql(name='expectancy', con=engine, if_exists='append', index=False)

In [18]:
# START OF THE SANDBOX TO TEST CODE

# I wanted to look at the dates when these restaurants were updated to see the spread of years
# I needed to split the string in the 'dateUpdated' column
date = fastfood_df['dateUpdated'].str.split('T', n=1, expand=True)
date[0].value_counts()

2019-04-01    567
2019-05-15    390
2019-05-12    383
2019-05-17    365
2019-04-18    341
             ... 
2019-01-08      1
2018-12-05      1
2019-01-02      1
2019-02-25      1
2019-01-07      1
Name: 0, Length: 144, dtype: int64

In [19]:
# I then added the data string onto the dataframe by creating a new column of the first list with the dates only
fastfood_df['Date'] = date[0]

In [48]:
# filter for only McDonald's fastfood restaurants
# isolate the 'Restaurants' with a conditional and store in a boolean variable
_mcds = fastfood_cols_df['Restaurant'] == "McDonald's"

# subset by applying the boolean variable to the dataframe. this will act as a filter
mcdonalds_df = fastfood_cols_df[_mcds]
# mcdonalds_df['Country'].value_counts()

In [60]:
mcdonalds_index_df = mcdonalds_df.groupby(['City']).count()
mcds = mcdonalds_index_df['Restaurant']
mcds_df = mcds.to_frame()
mcds_df

,Restaurant
City,
Abbeville,1
Aberdeen,1
Abingdon,1
Adairsville,1
Albany,1
...,...
York,2
Yuba City,1
Yukon,1


In [38]:
mcds_df.sort_values('Restaurant', ascending = False)

NameError: name 'mcds_df' is not defined

In [42]:
mcdonalds_df.groupby(['City']).count()

,Country,Restaurant,Date
City,,,
Abbeville,1,1,1
Aberdeen,1,1,1
Abingdon,1,1,1
Adairsville,1,1,1
Albany,1,1,1
...,...,...,...
York,2,2,2
Yuba City,1,1,1
Yukon,1,1,1


In [44]:
# pull the columns we want to use for our dataframe using double brackets to pull the dataframes [[]]
fastfood_cols_df = fastfood_df[['city', 'country', 'name', 'Date']]

# rename the columns
fastfood_cols_df.columns = ['City', 'Country', 'Restaurant', 'Date']
fastfood_cols_df

KeyError: "['Date'] not in index"